In [ ]:
## SCHEMA
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, DateTime, Text, Float, Boolean, JSON
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Define the base class for declarative models
Base = declarative_base()

# Define the Users table
class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    username = Column(String, unique=True, nullable=False)
    email = Column(String, unique=True, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)
    # Add relationship for cascade options
    chats = relationship("Chat", back_populates="user", cascade="all, delete-orphan")
    usage_records = relationship("ModelUsage", back_populates="user")

# Define the Chats table
class Chat(Base):
    __tablename__ = 'chats'
    
    chat_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('users.user_id', ondelete="CASCADE"), nullable=True)
    title = Column(String, default='New Chat')
    created_at = Column(DateTime, default=datetime.utcnow)
    # Add relationships for cascade options
    user = relationship("User", back_populates="chats")
    messages = relationship("Message", back_populates="chat", cascade="all, delete-orphan")
    usage_records = relationship("ModelUsage", back_populates="chat")

# Define the Messages table
class Message(Base):
    __tablename__ = 'messages'
    
    message_id = Column(Integer, primary_key=True, autoincrement=True)
    chat_id = Column(Integer, ForeignKey('chats.chat_id', ondelete="CASCADE"), nullable=False)
    sender = Column(String, nullable=False)  # 'user' or 'ai'
    content = Column(Text, nullable=False)
    timestamp = Column(DateTime, default=datetime.utcnow)
    # Add relationship for cascade options
    chat = relationship("Chat", back_populates="messages")
    feedback = relationship("MessageFeedback", back_populates="message", uselist=False, cascade="all, delete-orphan")

# Define the Model Usage table
class ModelUsage(Base):
    """Tracks AI model usage metrics for analytics and billing purposes."""
    __tablename__ = 'model_usage'
    
    usage_id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.user_id', ondelete="SET NULL"), nullable=True)
    chat_id = Column(Integer, ForeignKey('chats.chat_id', ondelete="SET NULL"), nullable=True)
    model_name = Column(String(100), nullable=False)
    provider = Column(String(50), nullable=False)
    prompt_tokens = Column(Integer, default=0)
    completion_tokens = Column(Integer, default=0)
    total_tokens = Column(Integer, default=0)
    query_size = Column(Integer, default=0)  # Size in characters
    response_size = Column(Integer, default=0)  # Size in characters
    cost = Column(Float, default=0.0)  # Cost in USD
    timestamp = Column(DateTime, default=datetime.utcnow)
    is_streaming = Column(Boolean, default=False)
    request_time_ms = Column(Integer, default=0)  # Request processing time in milliseconds
    # Add relationships
    user = relationship("User", back_populates="usage_records")
    chat = relationship("Chat", back_populates="usage_records")

# Define the Code Execution table
class CodeExecution(Base):
    """Tracks code execution attempts and results for analysis and debugging."""
    __tablename__ = 'code_executions'
    
    execution_id = Column(Integer, primary_key=True, autoincrement=True)
    message_id = Column(Integer, ForeignKey('messages.message_id', ondelete="CASCADE"), nullable=True)
    chat_id = Column(Integer, ForeignKey('chats.chat_id', ondelete="CASCADE"), nullable=True)
    user_id = Column(Integer, ForeignKey('users.user_id', ondelete="SET NULL"), nullable=True)
    
    # Code tracking
    initial_code = Column(Text, nullable=True)  # First version of code submitted
    latest_code = Column(Text, nullable=True)  # Most recent version of code
    
    # Execution results
    is_successful = Column(Boolean, default=False)
    output = Column(Text, nullable=True)  # Full output including errors
    
    # Model and agent information
    model_provider = Column(String(50), nullable=True)
    model_name = Column(String(100), nullable=True)
    model_temperature = Column(Float, nullable=True)
    model_max_tokens = Column(Integer, nullable=True)
    
    # Failure information
    failed_agents = Column(Text, nullable=True)  # JSON list of agent names that failed
    error_messages = Column(Text, nullable=True)  # JSON map of error messages by agent
    
    # Metadata
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
class MessageFeedback(Base):
    """Tracks user feedback and model settings for each message."""
    __tablename__ = 'message_feedback'
    
    feedback_id = Column(Integer, primary_key=True, autoincrement=True)
    message_id = Column(Integer, ForeignKey('messages.message_id', ondelete="CASCADE"), nullable=False)
    
    # User feedback
    rating = Column(Integer, nullable=True)  # Star rating (1-5)
    
    # Model settings used for this message
    model_name = Column(String(100), nullable=True)
    model_provider = Column(String(50), nullable=True)
    temperature = Column(Float, nullable=True)
    max_tokens = Column(Integer, nullable=True)

    # Metadata
    created_at = Column(DateTime, default=datetime.utcnow)
    updated_at = Column(DateTime, default=datetime.utcnow, onupdate=datetime.utcnow)
    
    # Relationship
    message = relationship("Message", back_populates="feedback")

In [1]:
import psycopg2
import pandas as pd

# Database connection parameters
db_params = {
    "host": "auto-analyst-db.cyfmmwg0gt9b.us-east-1.rds.amazonaws.com",
    "database": "autoanalystdb",
    "user": "postgres",
    "password": "L6sxfEJ2Dx87xNEGkA9y"
}

def run_query(query):
    """
    Executes a SQL query and returns the result as a pandas DataFrame.
    """
    try:
        with psycopg2.connect(**db_params) as conn:
            return pd.read_sql_query(query, conn)
    except Exception as e:
        print("Error executing query:", e)

# Example usage
query = "SELECT * FROM USERS"
df = run_query(query)

# non guest users where username doest not have guest in it
query = "SELECT * FROM USERS WHERE username NOT LIKE '%guest%'"
df = run_query(query)

print(df)
# count of non guest users
query = "SELECT COUNT(*) FROM USERS WHERE username NOT LIKE '%guest%'"
df = run_query(query)

print(df)


C:\Users\ashad\AppData\Local\Temp\ipykernel_10040\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


     user_id              username                             email  \
0          2         Ashad Qureshi               ashadq345@gmail.com   
1         12         Ashad qureshi              ashad001sp@gmail.com   
2         13         Arslan Shahid  arslan@firebird-technologies.com   
3         15              gdkvd _3               gdkvd2048@gmail.com   
4         55           Zain Sarwar         phicreative1997@gmail.com   
..       ...                   ...                               ...   
109      493  Дмитрий Колесниченко                 zeter11@gmail.com   
110      494                 Ден Д               ppachutin@gmail.com   
111      495       Askhat Urazbaev                  askhat@gmail.com   
112      499   Kaviradj Krishnadas      kaviradjkrishnadas@gmail.com   
113      500        Настя Куликова         nastya.kul.1703@gmail.com   

                    created_at  
0   2025-04-28 12:54:04.329562  
1   2025-04-28 19:29:46.374201  
2   2025-04-29 03:13:53.081492  
3  

C:\Users\ashad\AppData\Local\Temp\ipykernel_10040\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


   count
0    114


In [5]:
query = """SELECT u.username, u.email
FROM users u
LEFT JOIN chats c ON u.user_id = c.user_id
WHERE c.chat_id IS NULL;
"""

df = run_query(query)

print(df)




C:\Users\ashad\AppData\Local\Temp\ipykernel_15384\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


           username                       email
0    guest_4bf1ca45  guest_4bf1ca45@example.com
1    guest_23889f80  guest_23889f80@example.com
2    guest_f467f92a  guest_f467f92a@example.com
3    guest_0ca4ce97  guest_0ca4ce97@example.com
4    guest_05a682e7  guest_05a682e7@example.com
..              ...                         ...
359  guest_4d76fe69  guest_4d76fe69@example.com
360  guest_d0f07420  guest_d0f07420@example.com
361  guest_da82a77b  guest_da82a77b@example.com
362  guest_3b61e9e6  guest_3b61e9e6@example.com
363  guest_759ce6fd  guest_759ce6fd@example.com

[364 rows x 2 columns]


In [8]:
query = "SELECT COUNT(*) FROM USERS"
print(run_query(query))


C:\Users\ashad\AppData\Local\Temp\ipykernel_15384\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


   count
0    439


In [14]:
queries = {
    "model_usage_small_queries": """
        SELECT 
            usage_id, user_id, chat_id,
            query_size, response_size, total_tokens
        FROM model_usage
        WHERE query_size < 10
        ORDER BY timestamp DESC;
    """,

    "messages_basic": """
        SELECT 
            message_id, chat_id, sender, LENGTH(content) AS content_length, timestamp
        FROM messages
        ORDER BY timestamp DESC
        LIMIT 100;
    """,

    "chats_basic": """
        SELECT 
            chat_id, user_id, title, created_at
        FROM chats
        ORDER BY created_at DESC
        LIMIT 100;
    """
}

# Example usage
print(run_query(queries["model_usage_small_queries"]))
print(run_query(queries["messages_basic"]))
print(run_query(queries["chats_basic"]))


C:\Users\ashad\AppData\Local\Temp\ipykernel_15384\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


    usage_id  user_id  chat_id  query_size  response_size  total_tokens
0       1122      406    296.0           4             52            12
1       1107      397    292.0           4             52            12
2       1029      334    267.0           4             52            12
3        985      332    261.0           4             52            12
4        926      313    251.0           4             52            12
5        924      297    242.0           4             52            12
6        922      297    242.0           4             52            12
7        917      311    250.0           4             52            12
8        915      311    250.0           4             52            12
9        913      311    250.0           4             52            12
10       911      311    250.0           4             52            12
11       909      311    250.0           4             52            12
12       866      291    239.0           4             52       

C:\Users\ashad\AppData\Local\Temp\ipykernel_15384\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


    message_id  chat_id sender  content_length                  timestamp
0         1078      360     ai            7313 2025-05-25 08:02:20.836187
1         1077      360   user              44 2025-05-25 08:01:40.221171
2         1076      359     ai            3078 2025-05-25 07:57:34.598905
3         1075      359   user              35 2025-05-25 07:57:31.715095
4         1074      358     ai            3695 2025-05-25 07:22:11.706279
..         ...      ...    ...             ...                        ...
95         983      321     ai             141 2025-05-23 16:03:11.440771
96         982      321   user              15 2025-05-23 16:03:04.359967
97         981      320     ai             153 2025-05-23 15:58:33.625452
98         980      320   user              15 2025-05-23 15:58:23.179757
99         979      319     ai             119 2025-05-23 15:54:03.901142

[100 rows x 5 columns]


C:\Users\ashad\AppData\Local\Temp\ipykernel_15384\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


    chat_id  user_id                       title                 created_at
0       360    494.0              Trend Analysis 2025-05-25 08:01:39.978195
1       359    494.0  Data Visualization Request 2025-05-25 07:57:31.477531
2       358    492.0              Trend Analysis 2025-05-25 07:17:34.246120
3       357    492.0  Data Visualization Request 2025-05-25 07:13:42.278123
4       356    491.0               Data Insights 2025-05-25 06:34:33.813821
..      ...      ...                         ...                        ...
95      261    332.0               Обсуждение БД 2025-05-21 08:26:15.829362
96      260    300.0             Поиск Топонимов 2025-05-21 08:03:58.749727
97      258    328.0        Possible Connections 2025-05-21 07:39:01.992868
98      257    326.0            Анализ белорусов 2025-05-21 05:31:31.122865
99      256    324.0          Статистика покупок 2025-05-21 04:46:11.190074

[100 rows x 4 columns]


In [11]:
# Read recent code executions for user 500 and display key columns

query = """
    SELECT 
        created_at, 
        user_id, 
        model_name, 
        is_successful, 
        initial_code,
        latest_code,
        failed_agents,
        output
    FROM code_executions 
    WHERE user_id = 500
    ORDER BY created_at DESC
    LIMIT 10
"""
df = run_query(query)

# Print the results in a readable format
for idx, row in df.iterrows():
    print("-" * 40)
    print(f"Date: {row['created_at']}")
    print(f"User: {row['user_id']}")
    print(f"Model: {row['model_name']}")
    print(f"Initial Code: {row['initial_code']}")
    print(f"Latest Code: {row['latest_code']}")
    print(f'Output: {row["output"]}')
    print(f"Status: {'Success' if row['is_successful'] else 'Failed'}")
    print(f"Failed Agents: {row['failed_agents']}")
    print("-" * 40)


C:\Users\ashad\AppData\Local\Temp\ipykernel_10040\2926610153.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


----------------------------------------
Date: 2025-05-25 11:34:02.000333
User: 500
Model: claude-3-5-sonnet-latest
Initial Code: # planner_preprocessing_agent code start

# Create a copy of the original dataframe
processed_df = df.copy()

# Create analysis dictionary to store results
label_analysis = {
    'input_text': {
        'total_rows': len(processed_df),
        'starts_with_question': 0,
        'contains_question': 0,
        'contains_answer': 0
    },
    'target_text': {
        'total_rows': len(processed_df),
        'starts_with_answer': 0,
        'contains_question': 0,
        'contains_answer': 0
    }
}

# Analyze input_text
label_analysis['input_text']['starts_with_question'] = processed_df['input_text'].str.startswith('вопрос:').sum()
label_analysis['input_text']['contains_question'] = processed_df['input_text'].str.contains('вопрос:', regex=False).sum()
label_analysis['input_text']['contains_answer'] = processed_df['input_text'].str.contains('ответ:', regex=Fal